In [15]:
import pandas as pd
train_file = r"C:\Users\BHARGAVI\Downloads\project_data\ghc_train.csv"
test_file = r"C:\Users\BHARGAVI\Downloads\project_data\ghc_test.csv"

# Load CSV files into pandas DataFrames
train_df = pd.read_csv(train_file)
test_df=pd.read_csv(test_file)

# Preprocessing and Tokenizing for DL Models

In [19]:
import pandas as pd

# Function to map original labels to binary values
def map_labels(df):
    label_mapping = {'__label__0': 0, '__label__1': 1}
    if 'label' in df.columns:
        df['label'] = df['label'].map(label_mapping)
    return df

# Function to ensure text column is string type and handle missing values
def preprocess_text_column(df):
    df['text'] = df['text'].fillna('').astype(str)
    return df

# File paths
train_file = r"C:\Users\BHARGAVI\Downloads\project_data\ghc_train.csv"
test_file = r"C:\Users\BHARGAVI\Downloads\project_data\ghc_test.csv"

# Load CSV files into pandas DataFrames
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

# Apply functions to preprocess train_df and test_df
train_df = map_labels(train_df)
test_df = map_labels(test_df)

train_df = preprocess_text_column(train_df)
test_df = preprocess_text_column(test_df)


In [24]:
import pandas as pd

# Function to convert text column to lowercase
def convert_to_lower(df, text):
    df[text] = df[text].str.lower()
    return df

# File paths
train_file = r"C:\Users\BHARGAVI\Downloads\project_data\ghc_train.csv"
test_file = r"C:\Users\BHARGAVI\Downloads\project_data\ghc_test.csv"

# Load CSV files into pandas DataFrames
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

# Apply convert_to_lower function to 'text' column in train_df and test_df
train_df = convert_to_lower(train_df, 'text')
test_df = convert_to_lower(test_df, 'text')

# Example of using the modified train_df and test_df
print("Modified train_df:")
print(train_df.head())

print("\nModified test_df:")
print(test_df.head())


Modified train_df:
                                                text  hd  cv  vo  label
0  he most likely converted to islam due to his n...   0   0   0      0
1  so ford lied about being a psychologist. recor...   0   0   0      0
2     jobs. education. ending abuse of nation. ca43.   0   0   0      0
3  i share a lot of your values, & like many who ...   0   0   0      0
4  i am so ready to get back to blogging! www.ben...   0   0   0      0

Modified test_df:
                                                text  hd  cv  vo  label
0  https://www.youtube.com/watch?v=kacwpkaktak a ...   0   0   0      0
1  very nice! i tend to get tired of the constant...   0   0   0      0
2        watch today. https://circumcisionmovie.com/   0   0   0      0
3  " thinking venues " first color layer blocking...   0   0   0      0
4  what about death penalty for perpetrators  and...   0   0   0      0


In [26]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Function to tokenize text data
def tokenize_texts(train_texts, test_texts, num_words=None):
    tokenizer = Tokenizer(num_words=num_words)
    tokenizer.fit_on_texts(train_texts)
    
    X_train_seq = tokenizer.texts_to_sequences(train_texts)
    X_test_seq = tokenizer.texts_to_sequences(test_texts)
    
    return tokenizer, X_train_seq, X_test_seq

# Function to pad sequences
def pad_sequences_data(X_train_seq, X_test_seq, max_sequence_length):
    X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length)
    X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length)
    return X_train_padded, X_test_padded

# Function to extract labels
def extract_labels(train_data, test_data):
    y_train = train_data['label'].values
    y_test = test_data['label'].values
    return y_train, y_test


In [28]:
# File paths
train_file = r"C:\Users\BHARGAVI\Downloads\project_data\ghc_train.csv"
test_file = r"C:\Users\BHARGAVI\Downloads\project_data\ghc_test.csv"

# Load CSV files into pandas DataFrames
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

# Example usage
tokenizer, X_train_seq, X_test_seq = tokenize_texts(train_df['text'], test_df['text'], num_words=5000)
X_train_padded, X_test_padded = pad_sequences_data(X_train_seq, X_test_seq, max_sequence_length=100)
y_train, y_test = extract_labels(train_df, test_df)

# LSTM 

In [ ]:
# Example usage
num_words = 5000
max_sequence_length = 100

tokenizer, X_train_seq, X_test_seq = tokenize_texts(train_df['text'], test_df['text'], num_words=num_words)

X_train_padded, X_test_padded = pad_sequences_data(X_train_seq, X_test_seq, max_sequence_length=max_sequence_length)

y_train, y_test = extract_labels(train_df, test_df)

# Define the LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
history = model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# Predict the labels for test data
y_pred_prob = model.predict(X_test_padded)
y_pred = (y_pred_prob > 0.5).astype(int)

# Print the accuracy and classification report
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print('Classification Report:')
print(classification_report(y_test, y_pred))